# Limpieza de Datos

## Datos faltantes

Pandas provee un conjunto de métodos para trabajar con datos faltantes.
Los métodos reconocen como datos faltantes valores que pueden provenir de Numpy o de Python nativo. 

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

#### Detección de datos faltantes

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data.isnull()

El método isnull() devuelve una **máscara booleana** para la serie que indica los datos faltantes. 

In [ ]:
string_data = pd.Series([None, 'artichoke', np.nan, 'avocado'])
# El método reconoce también al valor faltante de Python nativo
string_data.isnull()

Para encontrar los valores con datos faltantes, podemos filtrar la serie utilizando boolean indexing

In [ ]:
# Filtro los valores nulos
print(string_data[string_data.isnull()])
print(' ')
# Filtro los valores no nulos
print(string_data[string_data.notnull()])

A la hora de trabajar con dataframes, podemos seleccionar las filas o columnas que no contienen ningún valor faltante 

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df

In [ ]:
# Ahora generamos algunos datos faltantes
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

In [ ]:
# Devuelve las filas completas
df.dropna()

In [ ]:
# Devuelve las columnas completas
df.dropna(axis=1)

#### Completar datos faltantes

In [ ]:
df.columns = ['col1','col2','col3']
df

In [ ]:
# Completar con un escalar
# Este método devuelve un nuevo objeto. Para modificar df directamente se utiliza el parámetro inplace=True
df.fillna(0)

In [ ]:
# Completar con un diccionario
df.fillna({'col2': 0.5, 'col3': -1})

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = np.nan 
df.iloc[4:, 2] = np.nan
df

In [ ]:
# Para completar en base a los últimos valores válidos, se puede utilizar el parámetro method = 'ffill'
df.fillna(method='ffill') 

#### Completar por la media y la media condicionada

El método fillna también acepta un nuevo dataframe con índices coincidentes con los valores faltantes. 

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                    'data2': np.random.rand(6)}, columns=['key', 'data1','data2'])
df

In [ ]:
df.iloc[2:3, 1] = np.nan
df.iloc[3:4, 2] = np.nan
df

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
df.fillna(df.mean())

In [ ]:
# Veamos las medias por grupo
print(df)
df.groupby('key').transform('mean')

In [ ]:
df.fillna(df.groupby('key').transform('mean'))

In [ ]:
df

##  Herramientas para manipulación de datos

Pandas cuenta con un conjunto de métodos que permiten operar sobre los elementos de un Dataframe o Serie.
Para aplicar la lógica deseada, podemos optar tanto por definir funciones con nombre como por utilizar expresiones lambda que luego no pueden reutilizarse.

    1)  pd.DataFrame.apply: Opera sobre filas o columnas completas
    2)  pd.DataFrame.applymap: Opera sobre cada uno de los elementos del Dataframe
    3)  pd.Series.apply: Opera sobre cada uno de los elementos de la Serie. 
    4)  pd.Series.map: Opera sobre cada uno de los elementos de la Serie, muy similar a Series.apply. 

La diferencia entre pd.Series.map y pd.Series.apply es que la segunda puede generar un Dataframe a partir de la serie, mientras que la primera si recibiera una serie como return de la función crearía una serie de series.

####  3.1 Función apply

La función apply de pandas permite realizar operaciones vectorizadas sobre los datasets tanto fila por fila como columna por columna.

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(5, 4), columns=['a', 'b', 'c', 'd'])
df

Utilizamos `df.apply` para encontrar la raíz cuadrada de los elementos de cada columna. `NaN` significa "Not a Number" y es el valor asignado a operaciones inválidas como la raíz de un número negativo.

El parámetro `axis = 0` es por default la fila, ese es el eje que se reduce.

In [ ]:
df.apply(np.sqrt)

In [ ]:
df.apply(np.mean)

**  Buscamos la media de todas las filas **

El parámetro axis=1 indica que la función se aplica para cada fila. Notar que el apply anterior no modificó el dataset, sino que creó una copia y luego modificó la misma. El dataset original conserva el mismo valor.

In [ ]:
df.apply(np.mean, axis=1)

`np.mean()` es una función que viene definida en numpy, pero podemos querer aplicar una función totalmente propia para, por ejemplo, crear una nueva columna que sea la suma entre las series a y d. Esto se puede hacer con expresiones lambda.

## Limpieza y Data transformation

Esta práctica se propone brindar un catálogo de métodos y funciones en Pandas y Pyhton que podrán ser útiles a la hora de encarar tareas de limpieza de datos. 

En general, podemos identificar seis tipos de tareas u operaciones que aplicamos a los datos en la etapa de limpieza.

1. Resolución de problemas de formato
2. Asignación de formatos adecuados
3. Corrección de valores erróneos
4. Estandarización de categorías
5. Imputación de datos faltantes (missing data imputation)
6. Organización correcta del dataset (tidy data)

Las funciones y métodos presentados abarcan una o varias de estas operaciones.

### Remover duplicados

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

* `duplicated()` devuelve un booleano identificando los casos duplicados.
* `drop_duplicates()` devuelve el `DataFrame` sin los casos duplicados

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data[~data.duplicated()] == data.drop_duplicates()

* Se puede utilizar `drop_duplicates()` para eliminar duplicados en una sola columna o en un set de columnas.

### Mapear y transformar los datos

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

* La idea es ahora poder asignar a cada `animal` una determinada `meat`. Una opción es hacerlo con los métodos `.map()

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

### Reemplazar valores

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

* Podemos hacer `replace` diferentes usando una lista de listas...

In [ ]:
data.replace([-999, -1000], [0, np.nan])

* ... O usando un `dict` 

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renombrar el índice de los ejes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])

In [ ]:
data.index.map(str.upper)

In [ ]:
data.index = data.index.map(str.upper)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
# Siempre devuelve una referencia al DataFrame, aunque no quiera utilizarla. Notar el nombre que se le asigna.

_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretizar y binarizar

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

* La función `cut` devuelve el intervalo abierto al que pertenece cada entrada

In [ ]:
#Defino el intervalo previamente

bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

* `codes` devuelve el indice del intervalo al que pertenece cada entrada

In [ ]:
cats.codes

In [ ]:
pd.value_counts(cats)

In [ ]:
pd.value_counts(cats.codes)

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

* ¿Qué diferencia observan con el objeto generado anteriormente?
* Es posible asignar nombres (etiquetas) a los intervalos generados. Puede hacerse a partir del parámetro `labels=`

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
# Divido en cuartiles

data = np.random.randn(1000)
cats = pd.qcut(data, 4) 
cats

In [ ]:
pd.value_counts(cats)

### Detectar y filtrar outliers

In [ ]:
np.random.seed(12345)
data = pd.DataFrame(np.random.randn(1000, 4))
data.head(5)

In [ ]:
col = data[3]
col[np.abs(col) > 3]

In [ ]:
data[~(np.abs(data) > 3).any(1)]

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

### Variables Dummies

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

In [ ]:
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

## Manipulación de strings

### String object methods

* `split()` toma un string, lo divide en función de un delimitador (`sep`) y devuelve una lista

In [ ]:
val = 'a,b,  guido'
val.split(',')

* `strip()` toma un string y devuelve un string sin los espacios iniciales y finales.

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

* `find()` devuelve el índice más bajo dentro de un string en el cual un substring es encontrado. Devuelve -1 si no la encuentra

In [ ]:
val.find(':')

* `index()` es similar, pero devuelve un `ValueError` cuando no encuentra el substring buscado

In [ ]:
val.index(',')

In [ ]:
val.index(':')

* `count()` cuenta la ocurrencia de un substring determinado en un string mayor.

In [ ]:
val.count(',')

* `replace()` reemplza un substring por otro.

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',', '')

### Regular expressions

In [ ]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

In [ ]:
regex = re.compile('\s+')
regex.split(text)

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

In [ ]:
print(regex.match(text))

In [ ]:
print(regex.sub('REDACTED', text))

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

In [ ]:
regex.findall(text)

In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

In [ ]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [ ]:
m = regex.match('wesm@bright.net')
m.groupdict()

### Funciones vectorizadas para strings en Pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)

In [ ]:
data

In [ ]:
data.isnull()

In [ ]:
data.str.contains('gmail')

In [ ]:
pattern = "\w"

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]